# Simulation Study for Noise Filtering

This is the v2 for the simulation study on the sparse jump model comparison with HMM, to show that SJM is able to filter away noisy data by using the weighting in the algorithm.


In [1]:
# Load Packages
import numpy as np
import pandas as pd
from hmmlearn.hmm import GaussianHMM
from sklearn.metrics import balanced_accuracy_score, confusion_matrix
from scipy.optimize import linear_sum_assignment
from joblib import Parallel, delayed
import multiprocessing
from scipy.stats import wilcoxon
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Jumpmodel-related
from jumpmodels.sparse_jump import SparseJumpModel
from jumpmodels.jump import JumpModel
from jumpmodels.preprocess import StandardScalerPD, DataClipperStd
from scipy import stats


## 1. Data Simulation & Utility Functions
def simulate_data(T, P, mu, random_state=None): """ Simulate data from a 2-state Gaussian HMM with correlation (tbd) between the features

In [2]:
def simulate_data(T, P, mu, random_state=None):
    """
    Simulate data from a 2-state Gaussian HMM with correlated noise for features beyond
    the first 15 informative features.
    """
    rng = np.random.default_rng(random_state)

    # Transition matrix for 2 states
    transmat = np.array([[0.9976, 0.0024],
                         [0.0232, 0.9768]])

    # Compute stationary distribution
    eigvals, eigvecs = np.linalg.eig(transmat.T)
    stat = np.real(eigvecs[:, np.isclose(eigvals, 1)])
    stat = stat[:, 0]
    stat = stat / np.sum(stat)

    # Generate state sequence
    states = np.zeros(T, dtype=int)
    states[0] = rng.choice(np.arange(2), p=stat)
    for t in range(1, T):
        states[t] = rng.choice(np.arange(2), p=transmat[states[t-1]])

    # Define means for each state: first 15 features are informative
    means = np.zeros((2, P))
    if P >= 15:
        means[0, :15] = mu
        means[1, :15] = -mu
    else:
        means[0, :P] = mu
        means[1, :P] = -mu

    # Prepare correlated noise for features beyond the first 15
    informative = 15
    if P > informative:
        num_noise = P - informative
        sigma = np.full((num_noise, num_noise), 0.213)
        np.fill_diagonal(sigma, 1.0)
        C = np.linalg.cholesky(sigma)
    else:
        C = None

    # Generate observations
    X = np.zeros((T, P))
    for t in range(T):
        n_inf = min(P, informative)
        X[t, :n_inf] = rng.normal(loc=means[states[t], :n_inf], scale=1.0, size=n_inf)
        if P > informative:
            noise_indep = rng.normal(loc=0.0, scale=1.0, size=P - informative)
            X[t, informative:] = C @ noise_indep
    return X, states

## 2.Aligning Predicted Labels With True Labels using the Hungarian Algorithm

In [3]:
def align_labels(true_labels, pred_labels):
    """
    Align predicted labels with true labels using the Hungarian algorithm.
    """
    D = confusion_matrix(true_labels, pred_labels)
    row_ind, col_ind = linear_sum_assignment(-D)
    mapping = {col: row for row, col in zip(row_ind, col_ind)}
    aligned = np.array([mapping[x] for x in pred_labels])
    return aligned

## 3. Setting up the function to calcuate the BAC

In [4]:
def calculate_bac(true_states, pred_states):
    """
    Compute balanced accuracy score after aligning predicted state labels.
    """
    aligned_pred = align_labels(true_states, pred_states)
    return balanced_accuracy_score(true_states, aligned_pred)

## 3.1 Computing per state accuracy and calculating statistic

In [5]:
def compute_per_state_accuracy(true_states, pred_states):
    """
    Compute per-state accuracy and BAC.
    """
    aligned = align_labels(true_states, pred_states)
    cm = confusion_matrix(true_states, aligned, labels=[0, 1])
    acc1 = cm[0, 0] / cm[0].sum() if cm[0].sum() > 0 else np.nan
    acc2 = cm[1, 1] / cm[1].sum() if cm[1].sum() > 0 else np.nan
    bac = (acc1 + acc2) / 2
    return acc1, acc2, bac

def compute_state_statistics(X, true_states, pred_states):
    """
    Compute mean for each state.
    """
    aligned = align_labels(true_states, pred_states)
    stats_dict = {}
    for state in [0, 1]:
        idx = np.where(aligned == state)[0]
        if len(idx) > 0:
            stats_dict[f'state{state}_mean'] = np.mean(X[idx])
        else:
            stats_dict[f'state{state}_mean'] = np.nan
    return stats_dict

## 3.2 Computing transition probabilities

In [6]:
def compute_transition_probabilities(state_seq):
    """
    Compute q12 and q21 from the state sequence.
    q12: transitions from state 0 to 1 / total transitions from state 0.
    q21: transitions from state 1 to 0 / total transitions from state 1.
    """
    state_seq = np.array(state_seq)
    prev = state_seq[:-1]
    nxt = state_seq[1:]
    if np.sum(prev == 0) > 0:
        q12 = np.sum((prev == 0) & (nxt == 1)) / np.sum(prev == 0)
    else:
        q12 = np.nan
    if np.sum(prev == 1) > 0:
        q21 = np.sum((prev == 1) & (nxt == 0)) / np.sum(prev == 1)
    else:
        q21 = np.nan
    return q12, q21

## 3.3 Standardizing features

In [7]:
def standardize_features(X):
    """
    Standardize features using the same procedure as in study 2.
    """
    X_df = pd.DataFrame(X)
    clipper = DataClipperStd(mul=3.0)
    scaler = StandardScalerPD()
    X_clipped = clipper.fit_transform(X_df)
    X_scaled = scaler.fit_transform(X_clipped)
    return X_scaled.values

## 4. Functions for model formulation

### 4.1 HMM With Nystrup (2020b) initialization

In [8]:
def run_mle(observations, n_components=2, init_type='default', seed=None):
    """
    Fit a Gaussian HMM to the data with different init_type.
    """
    model = GaussianHMM(n_components=n_components, covariance_type='diag', n_iter=100, random_state=seed)

    if init_type == 'default':
        model.startprob_ = np.array([1.0, 0.0])
        model.transmat_ = np.array([[0.9, 0.1],
                                    [0.1, 0.9]])
        model.means_ = np.zeros((n_components, observations.shape[1]))
        model.covars_ = np.full((n_components, observations.shape[1]), 1e-2)
        model.init_params = ''
    elif init_type == 'kmeans':
        kmeans = KMeans(n_clusters=n_components, n_init=10, random_state=seed)
        labels = kmeans.fit_predict(observations)
        means = []
        covars = []
        for i in range(n_components):
            obs_i = observations[labels == i]
            means.append(np.mean(obs_i, axis=0))
            covars.append(np.var(obs_i, axis=0) + 1e-2)
        model.startprob_ = np.ones(n_components) / n_components
        model.transmat_ = np.ones((n_components, n_components)) / n_components
        model.means_ = np.array(means)
        model.covars_ = np.array(covars)
        model.init_params = 'tmc'

    model.fit(observations)
    pred_states = model.predict(observations)
    return model, pred_states

def run_mle_default(observations, seed=None):
    return run_mle(observations, init_type='default', seed=seed)

def run_mle_kmeans(observations, seed=None):
    return run_mle(observations, init_type='kmeans', seed=seed)

### 4.2 Normal (Standard) Jump Model with Grid Search over λ

In [9]:
def run_jump_model_grid_search(X, true_states, n_components=2, random_state=None):
    """
    Grid search for the jump model over a range of lambda values.
    Uses set_params to avoid re-instantiating JumpModel each time.
    """
    lambda_values = np.logspace(-2, 4, 14)
    # Create one JumpModel instance (no jump_penalty at init)
    model = JumpModel(n_components=n_components, jump_penalty=0.0, cont=False,
                      max_iter=10, random_state=random_state)
    best_bac = -np.inf
    best_labels = None
    best_lambda = None
    grid_results = []

    for lam in lambda_values:
        # Update jump_penalty
        model.set_params(jump_penalty=lam)
        model.fit(X)
        labels = model.labels_
        bac = calculate_bac(true_states, labels)
        grid_results.append({'lambda': lam, 'BAC': bac})

        if bac > best_bac:
            best_bac = bac
            best_labels = labels
            best_lambda = lam

    return best_labels, best_bac, best_lambda, pd.DataFrame(grid_results)

### 4.3 Sparse Jump Model with Grid Search over λ and kappa

In [10]:
def run_sparse_jump_model_grid_search(X, true_states, n_components=2, random_state=None):
    """
    Grid search for the sparse jump model over combinations of lambda and kappa.
    Uses set_params to avoid re-instantiating SparseJumpModel each time.
    """
    lambdas = np.logspace(-1, 2, 7)
    p = X.shape[1]
    kappas = np.linspace(1, np.sqrt(p), 14)

    # Create one SparseJumpModel instance
    model = SparseJumpModel(n_components=n_components, jump_penalty=0.0, cont=False,
                            max_iter=10, random_state=random_state)

    best_bac = -np.inf
    best_labels = None
    best_lambda = None
    best_kappa = None
    grid_results = []

    for lam in lambdas:
        for kappa in kappas:
            max_feats = kappa**2
            # Update jump_penalty and max_feats
            model.set_params(jump_penalty=lam, max_feats=max_feats)
            model.fit(X)
            labels = model.labels_
            bac = calculate_bac(true_states, labels)

            grid_results.append({'lambda': lam, 'kappa': kappa,
                                 'max_feats': max_feats, 'BAC': bac})

            if bac > best_bac:
                best_bac = bac
                best_labels = labels
                best_lambda = lam
                best_kappa = kappa

    return best_labels, best_bac, best_lambda, best_kappa, pd.DataFrame(grid_results)

# 5. Main Execution
 We split the code into three sections for each model and then combine results at the end.


In [11]:
T = 500
mu_values = [0.02, 0.05, 0.1, 0.25, 0.5]
p_values = [15, 30, 60, 150, 300]
n_simulations = 100  # adjust as needed
n_components = 2

# Lists to collect simulation details and grid search results
simulation_results = []
grid_search_jump_list = []
grid_search_sparse_list = []

num_cores = multiprocessing.cpu_count()
print(f"Detected {num_cores} CPU cores for parallel processing.")

for mu in mu_values:
    for P in p_values:
        print(f"Running simulations for mu={mu} and P={P}...")
        # Parallelize simulations for this (mu, P)
        def simulate_and_run(sim):
            # Simulate once
            X, true_states = simulate_data(T, P, mu, random_state=sim)

            # MLE default and MLE_k
            model_mle, pred_mle = run_mle_default(X, seed=sim)
            model_mle_k, pred_mle_k = run_mle_kmeans(X, seed=sim)
            bac_mle = calculate_bac(true_states, pred_mle)
            bac_mle_k = calculate_bac(true_states, pred_mle_k)

            # Standardize features for jump models
            X_std = standardize_features(X)

            # Jump model grid search
            pred_jump, bac_jump, best_lambda_jump, df_grid_jump = run_jump_model_grid_search(
                X_std, true_states, n_components=n_components, random_state=sim)

            # Sparse jump model grid search
            pred_sparse, bac_sparse, best_lambda_sparse, best_kappa_sparse, df_grid_sparse = run_sparse_jump_model_grid_search(
                X_std, true_states, n_components=n_components, random_state=sim)

            # Compute performance metrics
            acc1_mle, acc2_mle, _ = compute_per_state_accuracy(true_states, pred_mle)
            acc1_mle_k, acc2_mle_k, _ = compute_per_state_accuracy(true_states, pred_mle_k)
            acc1_jump, acc2_jump, _ = compute_per_state_accuracy(true_states, pred_jump)
            acc1_sparse, acc2_sparse, _ = compute_per_state_accuracy(true_states, pred_sparse)

            stats_mle = compute_state_statistics(X, true_states, pred_mle)
            stats_mle_k = compute_state_statistics(X, true_states, pred_mle_k)
            stats_jump = compute_state_statistics(X, true_states, pred_jump)
            stats_sparse = compute_state_statistics(X, true_states, pred_sparse)

            q12_mle, q21_mle = compute_transition_probabilities(pred_mle)
            q12_mle_k, q21_mle_k = compute_transition_probabilities(pred_mle_k)
            q12_jump, q21_jump = compute_transition_probabilities(pred_jump)
            q12_sparse, q21_sparse = compute_transition_probabilities(pred_sparse)

            run_dict = {
                'mu': mu,
                'P': P,
                'sim': sim,
                # MLE (default)
                'MLE_BAC': bac_mle,
                'MLE_acc1': acc1_mle,
                'MLE_acc2': acc2_mle,
                'MLE_state0_mean': stats_mle.get('state0_mean', np.nan),
                'MLE_state1_mean': stats_mle.get('state1_mean', np.nan),
                'MLE_q12': q12_mle,
                'MLE_q21': q21_mle,
                # MLE_k (kmeans)
                'MLEK_BAC': bac_mle_k,
                'MLEK_acc1': acc1_mle_k,
                'MLEK_acc2': acc2_mle_k,
                'MLEK_state0_mean': stats_mle_k.get('state0_mean', np.nan),
                'MLEK_state1_mean': stats_mle_k.get('state1_mean', np.nan),
                'MLEK_q12': q12_mle_k,
                'MLEK_q21': q21_mle_k,
                # Jump Model
                'Jump_BAC': bac_jump,
                'Jump_acc1': acc1_jump,
                'Jump_acc2': acc2_jump,
                'Jump_state0_mean': stats_jump.get('state0_mean', np.nan),
                'Jump_state1_mean': stats_jump.get('state1_mean', np.nan),
                'Jump_q12': q12_jump,
                'Jump_q21': q21_jump,
                'Jump_best_lambda': best_lambda_jump,
                # Sparse Jump Model
                'SparseJump_BAC': bac_sparse,
                'SparseJump_acc1': acc1_sparse,
                'SparseJump_acc2': acc2_sparse,
                'SparseJump_state0_mean': stats_sparse.get('state0_mean', np.nan),
                'SparseJump_state1_mean': stats_sparse.get('state1_mean', np.nan),
                'SparseJump_q12': q12_sparse,
                'SparseJump_q21': q21_sparse,
                'SparseJump_best_lambda': best_lambda_sparse,
                'SparseJump_best_kappa': best_kappa_sparse
            }

            # Store grid search data frames with extra columns
            df_grid_jump['mu'] = mu
            df_grid_jump['P'] = P
            df_grid_jump['sim'] = sim

            df_grid_sparse['mu'] = mu
            df_grid_sparse['P'] = P
            df_grid_sparse['sim'] = sim

            return run_dict, df_grid_jump, df_grid_sparse

        results = Parallel(n_jobs=num_cores)(
            delayed(simulate_and_run)(sim) for sim in range(n_simulations)
        )

        for run_dict, df_grid_jump, df_grid_sparse in results:
            simulation_results.append(run_dict)
            grid_search_jump_list.append(df_grid_jump)
            grid_search_sparse_list.append(df_grid_sparse)

# Final DataFrames
df_simulation = pd.DataFrame(simulation_results)
df_grid_jump = pd.concat(grid_search_jump_list, ignore_index=True)
df_grid_sparse = pd.concat(grid_search_sparse_list, ignore_index=True)

print("Simulation results shape:", df_simulation.shape)
print("Jump grid search results shape:", df_grid_jump.shape)
print("Sparse Jump grid search results shape:", df_grid_sparse.shape)

Detected 16 CPU cores for parallel processing.
Running simulations for mu=0.02 and P=15...
Running simulations for mu=0.02 and P=30...
Running simulations for mu=0.02 and P=60...
Running simulations for mu=0.02 and P=150...
Running simulations for mu=0.02 and P=300...
Running simulations for mu=0.05 and P=15...
Running simulations for mu=0.05 and P=30...
Running simulations for mu=0.05 and P=60...
Running simulations for mu=0.05 and P=150...
Running simulations for mu=0.05 and P=300...
Running simulations for mu=0.1 and P=15...
Running simulations for mu=0.1 and P=30...
Running simulations for mu=0.1 and P=60...
Running simulations for mu=0.1 and P=150...
Running simulations for mu=0.1 and P=300...
Running simulations for mu=0.25 and P=15...
Running simulations for mu=0.25 and P=30...
Running simulations for mu=0.25 and P=60...
Running simulations for mu=0.25 and P=150...
Running simulations for mu=0.25 and P=300...
Running simulations for mu=0.5 and P=15...
Running simulations for mu=

## 6 Wilcoxon Test Table Generation

In [12]:
# Create a Wilcoxon table comparing Sparse Jump BAC versus the other models (MLE, MLEK, Jump)
wilcoxon_rows = []

for mu in mu_values:
    for P in p_values:
        df_subset = df_simulation[(df_simulation['mu'] == mu) & (df_simulation['P'] == P)]
        if len(df_subset) < 2:
            continue
        # Compare BAC: Sparse Jump vs MLE
        stat, p_sparse_mle = wilcoxon(df_subset['SparseJump_BAC'], df_subset['MLE_BAC'])
        # Compare BAC: Sparse Jump vs MLE_k
        stat, p_sparse_mlek = wilcoxon(df_subset['SparseJump_BAC'], df_subset['MLEK_BAC'])
        # Compare BAC: Sparse Jump vs Jump
        stat, p_sparse_jump = wilcoxon(df_subset['SparseJump_BAC'], df_subset['Jump_BAC'])

        wilcoxon_rows.append({
            'mu': mu,
            'P': P,
            'p_value_SparseJump_vs_MLE': p_sparse_mle,
            'p_value_SparseJump_vs_MLEK': p_sparse_mlek,
            'p_value_SparseJump_vs_Jump': p_sparse_jump
        })

df_wilcoxon = pd.DataFrame(wilcoxon_rows)
print("Wilcoxon test results (Sparse Jump vs other models):")
print(df_wilcoxon)

Wilcoxon test results (Sparse Jump vs other models):
      mu    P  p_value_SparseJump_vs_MLE  p_value_SparseJump_vs_MLEK  p_value_SparseJump_vs_Jump
0   0.02   15               1.767995e-10                4.095127e-17                2.969775e-02
1   0.02   30               3.895681e-18                3.895900e-18                8.390082e-04
2   0.02   60               3.895900e-18                3.895681e-18                8.636290e-09
3   0.02  150               3.895900e-18                3.895681e-18                1.392396e-10
4   0.02  300               3.896120e-18                3.896120e-18                1.714458e-11
5   0.05   15               2.345364e-10                5.129707e-17                2.162318e-02
6   0.05   30               3.895681e-18                3.895900e-18                7.329036e-04
7   0.05   60               3.895900e-18                3.895681e-18                5.626758e-09
8   0.05  150               3.895900e-18                3.895681e-18      

### 7 Summary Tables

In [13]:
def summarize_method(group, prefix):
    """
    Extracts the mean of relevant columns for a given method prefix.
    """
    return {
        'state0_mean': group[f'{prefix}_state0_mean'].mean(),
        'state1_mean': group[f'{prefix}_state1_mean'].mean(),
        'acc1':        group[f'{prefix}_acc1'].mean(),
        'acc2':        group[f'{prefix}_acc2'].mean(),
        'BAC':         group[f'{prefix}_BAC'].mean()
    }

grouped = df_simulation.groupby(['mu', 'P'])

for (m, p), group in grouped:
    row_mle   = summarize_method(group, 'MLE')
    row_mlek  = summarize_method(group, 'MLEK')
    row_jump  = summarize_method(group, 'Jump')
    row_sjump = summarize_method(group, 'SparseJump')

    print(f"\nSummary for mu={m} & P={p}")
    print("--------------------------------------------------------------------------")
    print("{:<12s}  {:>12s}  {:>12s}  {:>12s}  {:>12s}  {:>12s}".format(
        "", "state0_mean", "state1_mean", "acc1", "acc2", "BAC"
    ))

    def fmt_row(method_name, r):
        return "{:<12s}  {:>12.4f}  {:>12.4f}  {:>12.4f}  {:>12.4f}  {:>12.4f}".format(
            method_name,
            r['state0_mean'], r['state1_mean'], 
            r['acc1'], r['acc2'], r['BAC']
        )

    print(fmt_row("MLE def", row_mle))
    print(fmt_row("MLE k-means", row_mlek))
    print(fmt_row("Jump Model", row_jump))
    print(fmt_row("Sparse Jump", row_sjump))


Summary for mu=0.02 & P=15
--------------------------------------------------------------------------
               state0_mean   state1_mean          acc1          acc2           BAC
MLE def             0.0180        0.0212        0.6797        0.3615        0.5512
MLE k-means         0.0199        0.0032        0.6929        0.2977        0.5359
Jump Model          0.0238        0.0010        0.7533        0.4845        0.6679
Sparse Jump         0.0291       -0.0048        0.6794        0.6349        0.6919

Summary for mu=0.02 & P=30
--------------------------------------------------------------------------
               state0_mean   state1_mean          acc1          acc2           BAC
MLE def             0.0144        0.0060        0.5435        0.4920        0.5224
MLE k-means         0.0094        0.0116        0.5447        0.4810        0.5187
Jump Model          0.0151        0.0005        0.7518        0.4950        0.6714
Sparse Jump         0.0222       -0.0149       

### 7.1 statistical testing summary:

In [ ]:
# define the metrics for which we want to create error values
#   - state0_mean: true value is mu
#   - state1_mean: true value is -mu
#   - acc1, acc2, and BAC: we assume the ideal is 1.0 (or user-provided references)
metric_list = ["state0_mean", "state1_mean", "acc1", "acc2", "BAC"]
model_prefixes = ["MLEK", "Jump", "SparseJump"]

# create a copy
final_result_errors = df_simulation.copy()

def compute_ref_values(row):
    mu = row['mu']
    return {
        "state0_mean": mu,
        "state1_mean": -mu,
        # example references
        "acc1": 0.992246,
        "acc2": 0.921367,
        "BAC": 0.956806
    }

refs = final_result_errors.apply(compute_ref_values, axis=1)

for metric in metric_list:
    for prefix in model_prefixes:
        error_col = f"{prefix}_{metric}_error"
        final_result_errors[error_col] = final_result_errors.apply(
            lambda row: abs(row[f"{prefix}_{metric}"] - compute_ref_values(row)[metric]),
            axis=1
        )

grouped = final_result_errors.groupby(['mu', 'P'])

from scipy.stats import wilcoxon
import numpy as np

def two_sided_wilcoxon_p(diffs, atol=1e-10):
    diffs = np.array(diffs)
    diffs = diffs[~np.isnan(diffs)]
    if len(diffs) == 0:
        return 1.0
    if np.allclose(diffs, 0, atol=atol):
        return 1.0
    try:
        _, p = wilcoxon(diffs)
        return p
    except ValueError:
        return np.nan

def one_sided_pval(diffs, p_two_sided):
    if np.isnan(p_two_sided):
        return np.nan
    mean_diff = np.mean(diffs)
    if mean_diff > 0:
        return p_two_sided / 2
    else:
        return 1 - p_two_sided / 2

for (mu_val, P_val), group in grouped:
    table_rows = []
    for metric in metric_list:
        col_mlek = f"MLEK_{metric}_error"
        col_jump = f"Jump_{metric}_error"
        col_sparse = f"SparseJump_{metric}_error"

        e_mlek = group[col_mlek]
        e_jump = group[col_jump]
        e_sparse = group[col_sparse]

        mean_mlek = e_mlek.mean()
        mean_jump = e_jump.mean()
        mean_sparse = e_sparse.mean()

        # Sparse < Jump
        diff_sj = e_jump - e_sparse
        p2_sj = two_sided_wilcoxon_p(diff_sj)
        p1_sj = one_sided_pval(diff_sj, p2_sj)

        # Jump < MLEK
        diff_jm = e_mlek - e_jump
        p2_jm = two_sided_wilcoxon_p(diff_jm)
        p1_jm = one_sided_pval(diff_jm, p2_jm)

        # Sparse < MLEK
        diff_sm = e_mlek - e_sparse
        p2_sm = two_sided_wilcoxon_p(diff_sm)
        p1_sm = one_sided_pval(diff_sm, p2_sm)

        table_rows.append({
            "Metric": metric,
            "MLEK Mean Error": mean_mlek,
            "Jump Mean Error": mean_jump,
            "SparseJump Mean Error": mean_sparse,
            "p (Sparse < Jump)": p1_sj,
            "p (Jump < MLEK)": p1_jm,
            "p (Sparse < MLEK)": p1_sm
        })
    
    wilcoxon_table = pd.DataFrame(table_rows)
    print(f"\n=== Wilcoxon Test Results for mu = {mu_val} and P = {P_val} ===")
    print(wilcoxon_table.to_string(index=False, float_format='%.4f'))

In [ ]:
# Save DataFrames as CSV files if desired
df_simulation.to_csv("df_simulation.csv", index=False)
df_grid_jump.to_csv("df_grid_jump.csv", index=False)
df_grid_sparse.to_csv("df_grid_sparse.csv", index=False)
df_wilcoxon.to_csv("df_wilcoxon.csv", index=False)
final_result_errors.to_csv("final_result_errors.csv", index=False)

print("All DataFrames have been saved as CSV files in the current directory.")